# Data and Analysis Plan: College Basketball to NBA: Star Prediction
## Team-13

- Amaan Bhojani (bhojani.a@northeastern.edu)
- Jirawat Zhou (zhou.jir@northeastern.edu)
- Joshua Newstadt (newstadt.j@northeastern.edu)
- Jarrett Anderson (anderson.jar@northeastern.edu)


## Project Goal:
Our motivation for the project is to try and see if we can predict which current college basketball players will have successful NBA players. Even today, certain college players seem like they will be stars in the league and they end up being busts, we aim to shed some light as to whose skills will properly translate at the next level. 

## Overview:
We will scrape our data from two separate websites. 

#### Basketball Reference
The first site we will scrape is basketball-reference.com which will give us a [list of all Players](https://www.basketball-reference.com/leagues/NBA_2021_totals.html) for a given season which we can reference to find their [college stats](https://www.basketball-reference.com/players).

<img src="https://lh3.googleusercontent.com/zcmiuxLfVzrc1x2NzxZ4mbuQ7-2mMAMpgnx7-EPHwu0mZrW3_gnMZ3R2gLu6X8-8nP448wiK724nJdltPsGonYT8dRD-_0zd_U9KOLlDs0NT-NeEQffvXjTxxlihJGLPeeMXgAs4ECU=w2400" width=800px/>

From the list of players we can obtain:
- name
- url to specific player page

<img src="https://lh3.googleusercontent.com/LvO1eGKEqjfSLe_OZeXCA4zjDx6-ifVXGq9YeXSAc7b_Qf-lee-QMWrM-3RZpS_h1a0u8u-BXKGzrPFbtciSqm-YnrVbUHtJLk6yb7yLUMSkaaZsSxDPey71r_qjr3TqyyZWiZ-7EWM=w2400">

Upon visiting an individual player's page we can find their college stats.

<img src="https://lh3.googleusercontent.com/pk68MH39TLKlwvPJQTZnZPVHbiWC55S3LNfZrivqDA0RZQSDGYliGO-xOIfUpN6D4Er3Yz9a6B2-zgLVHbmA_XG4FukDrPaAjzzzcggQWBralbFjgkl7CEMersioOxbPrY-S4Z4ZplQ=w2400">

This is the table we are targeting as it contains the:
- season
- age
- college
- total stats for the season

Some players will have played multiple college seasons while other only one. To simplify our analysis, each player's college stats will be reduced to their total college career averages across every category.  


#### Fantasy Pros
The second site that we will scrape is fantasypros.com which will give us a [list of all Players including their Fantasy Points](https://www.fantasypros.com/nba/stats/overall.php) for a given season.

<img src="https://lh3.googleusercontent.com/kwNP7yD1RCmmBpr7ws4jNgE_AY1JV4TFnacpKohGnlt9bn-JlL5ujDX0aEWubOrXgDeP1wcfTuhaZ0QVuuAKJkeOkSe4Yud2Z-yszUzAfvOhWvsC_uVS4ahGFjO-p8A6-dxa5Lx9py4=w2400">

This table is targeted as it provides each NBA player's total accrued Fantasy Points for a given season.

## Pipeline Overview
We will accomplish this task by creating various functions. 

#### Basketball Reference
To get each player's total college career stats we will use eight functions:
- `get_nba_player_html(year)`
    - returns the raw html webscrapped from [basketball-reference.com](https://www.basketball-reference.com/leagues/NBA_2021_totals.html) 
- `extract_nba_player(html_player)` 
    - returns a DataFrame of the webscrapped NBA Player's and their respective NBA Stat Page URLs 
- `get_nba_collegestat(player_url)`
    - returns the raw html webscrapped from the NBA Player's Stat Page
- `extract_nba_collegestat(html_stat)`
    - returns a Dictionary of the NBA Player's College Stats by season
- `build_nba_college(df_player)`
    - returns a DataFrame of the all the NBA Player's College Stats by season
- `get_multiple_years(df_college_stats)`
    -  returns a GroupBy Object representing the grouping of an NBA Player and their individual seasons of college play
- `avg_college_stats(df_multiples)`
    - returns a DataFrame of an NBA player and their Average College Career Stats
- `build_final_college(df_college_avg, df_college_stats)`
    - returns all the college stats, one player per fow, multi-year stats averaged 
    
#### Fantasy Pros
To get each player's total fantasy points we will use 2 functions:
- `get_fantasy_stat(year)`
    - returns the raw html webscraped from [fantasypros.com](https://www.fantasypros.com/nba/stats/overall.php)
- `extract_nba_stat(nba_html)`
    - returns a DataFrame of NBA Player's Total Fantasy Points and Season Stats

As well as three scripts:
- **Scrape List of Player's and their College Stats:** use `get_nba_player(year)` and `extract_nba_player(html_player)` to get all the NBA Players and their Player Page URLs, populating a DataFrame `df_player`. From there use `build_nba_college(df_player)` on the DataFrame `df_player`, which populates a DataFrame `df_college_stats`. 
- **Clean the College Data:**  Use `get_multiple_years(df_college_stats)` on `df_college_stats`, which populates a DataFrame `df_multiples`. From there use `avg_college_stats(df_multiples)` on `df_multiples`, populating a DataFrame `df_college_avg`. Then we use `build_final_college(df_college_avg, df_college_stats)` on the `df_college_avg` and `df_college_stats` DataFrames. This reduces these two DataFrames, populating a new DataFrame `df_final_college`.
- **Scrape List of Player's and their Fantasy Points:** use `get_fantasy_stat(year)` and `extract_nba_stat(nba_html)` populating a DataFrame `df_nba_stat`
- **Combine Player's Fantasy Points and their Average College Stats:** use `get_fantasy_points(df_nba_stat, df_final_college)` on `df_nba_stat` and `df_final_college` to get the fantasy poitns for only players with college stats, populating a Dictionary `fantasy_dict`. Then use `build_all_stats(fantasy_dict, df_final_college)` on `fantasy_dict` and `df_final_college` to build a DataFrame of all college stats and fantasy points. Save this new DataFrame as `df_all_stats`. 

## Pipeline

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

### Basketball Reference

In [2]:
def get_nba_player_html(year):
    """ Web scrapping https://www.basketball-reference.com/leagues/NBA_2021_totals.html
        to retrieve NBA player college statistic
        
        Args:
            year (int) : Represent year in yyyy format (e.g. 2021)
        
        Return:
            html_player (html) : raw html webscrapped from basketball-reference.com
    """
    nba_player = f'https://www.basketball-reference.com/leagues/NBA_{year}_totals.html'
    html_player = requests.get(nba_player)
    status = html_player.status_code
    
    # If page load correctly
    if status == 200:
        return html_player.text

In [3]:
def extract_nba_player(html_player):
    """ Extract the NBA player from the crawled webpage in order to retrieve stat
    
        Args:
            html_player (String) : Represent the crawled league webpage
            
        Return:
            df_player (DataFrame) : Represent list of player and corresponding stat URL
    """
    base_url = 'https://www.basketball-reference.com'
    soup = BeautifulSoup(html_player)
    df_player = pd.DataFrame()
    
    # Find all player with stat URL
    for player in soup.find_all('td', {'data-stat': 'player'}):
        link = player.find_all('a')[0]
        
        # Construct Dict that contain Player Name and URL to player stat
        dict_player = {'player_name' : player.text,
                      'url' : base_url + link.attrs['href']}
        
        df_player = df_player.append(dict_player, ignore_index=True)
        
    
    # Drop any duplicate and retain the first entries
    df_player.drop_duplicates(subset='player_name', keep='first', inplace=True)
        
    return df_player

In [4]:
def get_nba_collegestat(player_url):
    """ Web Scrapping https://www.basketball-reference.com/players and retrive player's
        college statistic
        
        Args:
            player_url (String) : Represent URL that contain player stat
            
        Return:
            html_stat (String) : Represent the stat for all NBA player in HTML
    """
    html_stat = requests.get(player_url)
    status = html_stat.status_code
    
    # If page load correctly
    if status == 200:
        return html_stat.text

In [5]:
def extract_nba_collegestat(html_stat):
    """ Extract college stat from NBA player
    
        Args:
            html_stat (String) : Represent the crawled player stat webpage
            
        Return:
            dict_stat (Dictionary) : Represent the player college stat
    """
    soup = BeautifulSoup(html_stat)
    
    # Clean up hidden scripts
    stat_wcomment = soup.find_all('div', {'id':'all_all_college_stats'})
    
    if not stat_wcomment:
        dict_stat = {}
        return dict_stat
    else:
        str_stat = str(stat_wcomment[0]).replace('<!--','').replace('-->','')
    
        soup = BeautifulSoup(str_stat)
        stat_body = soup.find('tbody')

        # Initalize Dict
        dict_stat = {}

        # Add year
        for year in stat_body.find_all('th'):
            # Get Year and Value
            key = year.get('data-stat')
            value = year.text

            if key not in dict_stat.keys():
                dict_stat[key] = [value]
            else:
                dict_stat[key] = dict_stat[key] + [value]


        # Add Stat Information
        for stat in stat_body.find_all('td'):
            # Get Metric and Value
            key = stat.get('data-stat')
            value = stat.text

            if key not in dict_stat.keys():
                dict_stat[key] = [value]
            else:
                dict_stat[key] = dict_stat[key] + [value]
    
        return dict_stat

In [6]:
def build_nba_college(df_player):
    """ Build the NBA college Data given list of player and URL
    
        Args:
            df_player (DataFrame) : Represent list of Player and corresponding stat URL
            
        Return:
            df_nba_collegestat (DataFrame) : Represent the NBA Player college stat
    """
    df_nba_collegestat = pd.DataFrame()
    
    # iterate through each player and url in dataframe
    for idx, row in df_player.iterrows():
        player = row['player_name']
        url = row['url']
        
        # get player stats as dictionary
        player_stat_url = get_nba_collegestat(url)
        dict_player_stat = extract_nba_collegestat(player_stat_url)
        
        # convert dictionary to dataframe
        df_player_temp = pd.DataFrame(dict_player_stat)
        df_player_temp['player_name'] = player
        
        # adds player dataframe to full stats dataframe
        df_nba_collegestat = df_nba_collegestat.append(df_player_temp)
        
    return df_nba_collegestat 

In [7]:
def get_multiple_years(df_college_stats):
    """ Gets dataframe of players who played multiple years

        Args:
            df_college_stats (pd.DataFrame): all college player statistics
        
        Returns:
            df_multiples (pd.GroupBy): all statistics of multiple year players
                                            grouped by player
    """
    # removes duplicates and groups them by player
    x = df_college_stats.duplicated(['player_name'],keep=False)
    df_multiples = df_college_stats[x].groupby('player_name',sort=False)
    
    return(df_multiples)

In [8]:
def avg_college_stats(df_multiples):
    """ Gets average of college career stats
    
    Args:
        df_multiples (pd.GroupBy): all statistics of multiple year players
                                            grouped by player
    
    Returns:
        df_college_avg (pd.DataFrame): average of each multi-year player's college career
    """
    # the columns names of averagable stats
    mean_column_list = list(df_college_stats.columns)[3:-1]
    
    # initalize list to collect each Series
    all_college_stats = []
    
    # iterates through players that have multi-year college careers
    for player in df_multiples:
        name, stats = player[0], player[1]
        
        # creates series of just averagable stats
        stats_to_avg = stats.loc[:,mean_column_list]
        
        # removes potential errors
        stats_to_avg.replace('',0.0,inplace=True)
        
        # converts strings to floats
        for column in stats_to_avg.columns:
            for idx in stats_to_avg[column].index: 
                stats_to_avg[column][idx] = float(stats_to_avg[column][idx])
        
        # averages college stats
        avg_stats = round(stats_to_avg.mean(axis=0), 2)

        # creates new career duration data
        career_season = stats['season'].iloc[0][:4]+'-'+stats['season'].iloc[-1][:4]
        
        # adds career duration and player name to series
        avg_stats['season'] = career_season
        avg_stats['player_name'] = name
        
        # appends series to list
        all_college_stats.append(avg_stats)
        
    # builds dataframe from list
    df_college_avg = pd.DataFrame(all_college_stats)
    
    return(df_college_avg)

In [9]:
def build_final_college(df_college_avg, df_college_stats):
    """ Combines dataframes of single year players with avg multi-year players
    
    Args:
        df_college_avg (pd.Dataframe): averaged stats of multi-year players
        df_college_stats (pd.DataFrame): stats for all players, each row a year
        
    Returns:
        df_final_college (pd.DataFrame): all college stats, one player per row
                                            multi-year stats averaged
    """
    # get multi-year players
    multiples = df_college_stats.duplicated(['player_name'],keep=False)
    
    # keeps just single year players
    df_college = df_college_stats[~multiples].drop(['college_id','age'],axis=1)
    
    # append averaged multi-years to dataframe of single years
    df_final_college = df_college.append(df_college_avg)
    
    # cleans indicies
    df_final_college.reset_index(inplace=True)
    df_final_college.drop('index',axis=1,inplace=True)
    
    return(df_final_college)

#### Scrape List of Player's and their College Stats:

In [10]:
html_player = get_nba_player_html(2021)
df_player = extract_nba_player(html_player)

In [11]:
df_player.head()

,player_name,url
0,Precious Achiuwa,https://www.basketball-reference.com/players/a...
1,Jaylen Adams,https://www.basketball-reference.com/players/a...
2,Steven Adams,https://www.basketball-reference.com/players/a...
3,Bam Adebayo,https://www.basketball-reference.com/players/a...
4,LaMarcus Aldridge,https://www.basketball-reference.com/players/a...


In [12]:
url = df_player.loc[0,'url']
html_stat = get_nba_collegestat(url)
extract_nba_collegestat(html_stat)

{'season': ['2019-20'],
 'age': ['20'],
 'college_id': ['MEMPHIS'],
 'g': ['31'],
 'mp': ['943'],
 'fg': ['182'],
 'fga': ['369'],
 'fg3': ['13'],
 'fg3a': ['40'],
 'ft': ['112'],
 'fta': ['187'],
 'orb': ['93'],
 'trb': ['334'],
 'ast': ['30'],
 'stl': ['34'],
 'blk': ['58'],
 'tov': ['87'],
 'pf': ['73'],
 'pts': ['489'],
 'fg_pct': ['.493'],
 'fg3_pct': ['.325'],
 'ft_pct': ['.599'],
 'mp_per_g': ['30.4'],
 'pts_per_g': ['15.8'],
 'trb_per_g': ['10.8'],
 'ast_per_g': ['1.0']}

In [13]:
df_college_stats = build_nba_college(df_player)

In [14]:
df_college_stats.head()

,season,age,college_id,g,mp,fg,fga,fg3,fg3a,ft,...,pf,pts,fg_pct,fg3_pct,ft_pct,mp_per_g,pts_per_g,trb_per_g,ast_per_g,player_name
0,2019-20,20,MEMPHIS,31,943,182,369,13,40,112,...,73,489,.493,.325,.599,30.4,15.8,10.8,1.0,Precious Achiuwa
0,2014-15,18,STBONNY,22,714,61,158,34,105,65,...,47,221,.386,.324,.783,32.5,10.0,2.5,4.5,Jaylen Adams
1,2015-16,19,STBONNY,30,1125,150,337,84,192,153,...,80,537,.445,.438,.874,37.5,17.9,3.7,5.0,Jaylen Adams
2,2016-17,20,STBONNY,30,1123,168,401,77,216,206,...,84,619,.419,.356,.821,37.4,20.6,3.7,6.5,Jaylen Adams
3,2017-18,21,STBONNY,28,1036,153,350,75,172,154,...,80,535,.437,.436,.851,37.0,19.1,3.4,5.2,Jaylen Adams


#### Clean the College Data:

In [15]:
df_multiples = get_multiple_years(df_college_stats)
df_multiples.head()

,season,age,college_id,g,mp,fg,fga,fg3,fg3a,ft,...,pf,pts,fg_pct,fg3_pct,ft_pct,mp_per_g,pts_per_g,trb_per_g,ast_per_g,player_name
0,2014-15,18,STBONNY,22,714,61,158,34,105,65,...,47,221,.386,.324,.783,32.5,10.0,2.5,4.5,Jaylen Adams
1,2015-16,19,STBONNY,30,1125,150,337,84,192,153,...,80,537,.445,.438,.874,37.5,17.9,3.7,5.0,Jaylen Adams
2,2016-17,20,STBONNY,30,1123,168,401,77,216,206,...,84,619,.419,.356,.821,37.4,20.6,3.7,6.5,Jaylen Adams
3,2017-18,21,STBONNY,28,1036,153,350,75,172,154,...,80,535,.437,.436,.851,37.0,19.1,3.4,5.2,Jaylen Adams
0,2004-05,19,TEXAS,16,355,57,86,0,0,44,...,,158,.663,,.657,22.2,9.9,5.9,0.9,LaMarcus Aldridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2019-20,20,MISSST,31,1025,136,275,30,70,50,...,58,352,.495,.429,.641,33.1,11.4,6.5,1.3,Robert Woodard II
0,2013-14,21,UTAH,33,1202,174,310,12,54,153,...,61,513,.561,.222,.793,36.4,15.5,6.8,5.3,Delon Wright
1,2014-15,22,UTAH,35,1165,165,324,26,73,153,...,49,509,.509,.356,.836,33.3,14.5,4.9,5.1,Delon Wright
0,2011-12,19,INDIANA,36,1025,200,321,0,0,163,...,97,563,.623,,.755,28.5,15.6,6.6,1.3,Cody Zeller


In [16]:
df_college_avg = avg_college_stats(df_multiples)
df_college_avg.head()

C:\Users\Jarrett\AppData\Local\Temp/ipykernel_46452/1339992317.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_to_avg[column][idx] = float(stats_to_avg[column][idx])


,g,mp,fg,fga,fg3,fg3a,ft,fta,orb,trb,...,pts,fg_pct,fg3_pct,ft_pct,mp_per_g,pts_per_g,trb_per_g,ast_per_g,season,player_name
0,27.50,999.50,133.00,311.50,67.50,171.25,144.50,172.50,14.75,93.0,...,478.00,0.42,0.39,0.83,36.10,16.90,3.32,5.30,2014-2017,Jaylen Adams
1,26.50,801.00,138.00,235.50,0.00,0.00,80.50,124.00,0.00,217.0,...,356.50,0.62,0.00,0.65,27.95,12.45,7.55,0.70,2004-2005,LaMarcus Aldridge
2,32.67,922.67,133.33,319.00,70.00,188.33,77.00,94.67,10.33,120.0,...,413.67,0.42,0.37,0.79,28.33,12.70,3.70,2.27,2017-2019,Ty-Shon Alexander
3,33.50,1003.00,157.00,338.50,58.00,151.50,79.00,103.50,19.50,133.0,...,451.00,0.46,0.38,0.75,29.85,13.45,3.95,2.75,2017-2018,Nickeil Alexander-Walker
4,35.50,991.25,151.50,352.25,72.75,191.50,123.25,147.75,19.00,112.5,...,499.00,0.43,0.37,0.84,27.75,14.00,3.15,3.00,2014-2017,Grayson Allen


In [17]:
df_final_college = build_final_college(df_college_avg, df_college_stats)
df_final_college.head()

,season,g,mp,fg,fga,fg3,fg3a,ft,fta,orb,...,pf,pts,fg_pct,fg3_pct,ft_pct,mp_per_g,pts_per_g,trb_per_g,ast_per_g,player_name
0,2019-20,31,943,182,369,13,40,112,187,93,...,73,489,.493,.325,.599,30.4,15.8,10.8,1.0,Precious Achiuwa
1,2012-13,32,749,100,175,0,0,31,70,90,...,52,231,.571,,.443,23.4,7.2,6.3,0.6,Steven Adams
2,2016-17,38,1145,170,284,0,0,154,236,118,...,99,494,.599,,.653,30.1,13.0,8.0,0.8,Bam Adebayo
3,2016-17,33,1061,179,316,0,7,84,149,100,...,68,442,.566,.000,.564,32.2,13.4,8.4,0.8,Jarrett Allen
4,2017-18,29,438,58,101,2,15,33,64,24,...,66,151,.574,.133,.516,15.1,5.2,2.9,0.4,Kostas Antetokounmpo


### Fantasy Pros

In [18]:
def get_fantasy_stat(year):
    """ Get all NBA Player Stat from https://www.fantasypros.com/nba/stats/overall.php
    
        Args:
            year (int) : Represent year in yyyy which NBA data will be extracted 
            
        Return:
            nba_html (string) : Represent the NBA stat in HTML Representation
    """
    nba_url = f'https://www.fantasypros.com/nba/stats/overall.php?year={year}'
    nba_html = requests.get(nba_url)
    status = nba_html.status_code
    
    # If page load correctly
    if status == 200:
        return nba_html.text

In [19]:
def extract_nba_stat(nba_html):
    """ Extract NBA Stat from the fantasypros website
    
        Args:
            nba_html (string) : Represent the NBA stat in HTML Representation
            
        Return:
            df_nba_stat (DataFrame) : Represent the dataframe contain NBA stat
    
    """
    dict_nba_stat = {'Player' : [], 
                    'PTS' : [],
                    'REB' : [],
                    'AST' : [],
                    'BLK' : [],
                    'STL' : [],
                    'FG%' : [],
                    'FT%' : [],
                    '3PM' : [],
                    'TO' : [],
                    'GP' : [],
                    'MIN' : [],
                    'FTM' : [],
                    '2PM' :[],
                    'A/TO' : [],
                    'PF':[]}
    df_nba_stat = pd.DataFrame(dict_nba_stat)
    soup = BeautifulSoup(nba_html)
    
    # Extract all Information
    for row in soup.find('tbody').find_all('tr'):
        arr = np.array([i.text for i in row.find_all('td')])
        
        df_nba_stat = df_nba_stat.append(pd.DataFrame(arr.reshape(1,-1), columns=list(df_nba_stat)), ignore_index=True)
        
    return df_nba_stat

#### Scrape List of Player's and their Fantasy Points:

In [20]:
nba_html = get_fantasy_stat(2021)
df_nba_stat = extract_nba_stat(nba_html)
df_nba_stat.head()

,Player,PTS,REB,AST,BLK,STL,FG%,FT%,3PM,TO,GP,MIN,FTM,2PM,A/TO,PF
0,"DeMar DeRozan (CHI - SF,PF,SG) DTD","2,019",384,360,24,64,.506,.874,48,169,72,"2,605",493,691,2.13,168
1,Trae Young (ATL - PG) DTD,"2,000",265,683,7,72,.460,.904,216,282,71,"2,472",462,445,2.42,117
2,"Jayson Tatum (BOS - SF,PF) DTD","1,946",578,310,45,69,.451,.860,219,207,72,"2,608",381,454,1.50,168
3,"Giannis Antetokounmpo (MIL - PF,C) DTD","1,897",735,366,90,66,.555,.723,69,209,63,"2,075",528,581,1.75,202
4,"Joel Embiid (PHI - PF,C) DTD","1,890",721,267,91,73,.491,.818,85,194,63,"2,114",605,515,1.38,171


#### Combine Player's Fantasy Points and their Average College Stats:

In [21]:
def get_fantasy_points(df_nba_stat, df_final_college):
    """ Gets fantasy points for only players with college stats
    
    Args: 
        df_nba_stat (pd.Dataframe): Dataframe contains NBA stat and fantasy points
        df_final_college (pd.Dataframe): all college stats, one player per row
        
    Returns:
        fantasy_dict (Dictionary): Player name as keys, fantasy points as values
    """
    # initalize dictionary to collect fantasy points
    fantasy_dict = {}
    
    # itterates through NBA stat dataframe
    for idx in df_nba_stat.index:
        
        # collects player name as keys and fantasy points as values
        name = df_nba_stat.loc[idx,'Player'].split('(')[0].strip()
        points = df_nba_stat.loc[idx,'PTS']
    
        # doesn't include players missing college data
        if name in list(df_final_college['player_name']):
            fantasy_dict[name] = points
    
    return(fantasy_dict)

In [22]:
fantasy_dict = get_fantasy_points(df_nba_stat, df_final_college)

In [23]:
def build_all_stats(fantasy_dict, df_final_college):
    """ Builds dataframe of all college stats and fantasy points
    
    Args:
        fantasy_dict (Dictionary): Player name as keys, fantasy points as values
        df_final_college (pd.Dataframe): all college stats, one player per row
        
    Returns:
        df_all_stats (pd.Dataframe): Dataframe of college stats and fantasy points
                                        one player per row
    """

    df_college_temp = df_final_college.set_index('player_name')
    
    # drop players that don't have fantasy point stats
    for name in df_college_temp.index:
        if name not in list(fantasy_dict.keys()):
            df_college_temp = df_college_temp.drop(name,axis=0)
    
    # initalize list for Series of player
    final_stats_list = []
    
    # itterate through college data
    for name in df_college_temp.index:
        
        # create series of player stats and add fantasy points
        player_series = df_college_temp.loc[name,:]
        player_series['f_PTS'] = fantasy_dict[name]
        
        # append to list of player series
        final_stats_list.append(player_series)
    
    # assemble dataframe from list
    df_all_stats = pd.DataFrame(final_stats_list)
    
    return(df_all_stats)

In [24]:
df_all_stats = build_all_stats(fantasy_dict, df_final_college)
df_all_stats.head()

,season,g,mp,fg,fga,fg3,fg3a,ft,fta,orb,...,pf,pts,fg_pct,fg3_pct,ft_pct,mp_per_g,pts_per_g,trb_per_g,ast_per_g,f_PTS
Precious Achiuwa,2019-20,31,943,182,369,13,40,112,187,93,...,73,489,.493,.325,.599,30.4,15.8,10.8,1.0,602
Steven Adams,2012-13,32,749,100,175,0,0,31,70,90,...,52,231,.571,,.443,23.4,7.2,6.3,0.6,519
Bam Adebayo,2016-17,38,1145,170,284,0,0,154,236,118,...,99,494,.599,,.653,30.1,13.0,8.0,0.8,990
Jarrett Allen,2016-17,33,1061,179,316,0,7,84,149,100,...,68,442,.566,.000,.564,32.2,13.4,8.4,0.8,904
Carmelo Anthony,2002-03,35,1274,277,612,56,166,168,238,,...,77,778,.453,.337,.706,36.4,22.2,10.0,2.2,892


In [25]:
### Run to save to .csv
df_all_stats.replace('','NaN').to_csv('nba_player_stats.csv')

## Visualizations
Our graphs were made using Tableau.

#### Current NBA Players and Their College Stats

<img src="https://lh3.googleusercontent.com/uzzGQLVkKpgItQLyAUFSJSx2E1aGN0u5KEAlM9F044JgsbTUgUI3ZE-xeqXsmgtdNDOJRW16pvC3NBE36Syg3y32Hl-G3Pb9pWADPt2Lz975ovS9sjMfApX8RccfU_k6_lFNRm9lQKY=w2400">


The graph above shows a snippet of current NBA and their main 3 stats tracked in college- points, rebounds, and assists (all per game). 

#### Current Season NBA Performance per Draft Class
<img src="https://lh3.googleusercontent.com/KKuTl7txziLdD--A7sgcC2RQVw7Nk5vPhLCTP9sAyL7r6XqUmuO0WCPliWVEi4efcqBGPoumVV764GFpWkBxLQGwM07iYMn30r22D0Dv6nZk7wD2AkH0TfLm6fcAwfUHx1KCk1vtmcc=w2400">

The graph above shows how each draft class has been performing as a whole this NBA season. Performance is based on fantasy points on the season and done by averaging the total fantasy points per player in each class.



## Analysis Plan
Our expectation is that players with similar college stats might express similar stats and fanatasy stat totals in the NBA. 

One of the algorithms that will be used in this project to predict future NBA stars is the K-Nearest-Neighbor algorithm. Using this algorithm we will predict if a new college basketball player is likely to become the next NBA star. Our training data will be players college career stats and their associated NBA Fantasy Points. In order to use the KNN classifier, we must first examine the importance of each features, or basically what are the features of all NBA star have in common, and which features are irrelevant in prediction. Hence, a random forest algorithm may be used to help us identify those common features between all NBA player during their college career, in order to come up with more accurate KNN classifer.

Another ML tools that we will be using is the confusion matrix and Kfold cross-validation algorithm, which will be used to evaluate the accuracy of the classifier. To evaluate the performance of our KNN, we will be calculating accuracy per group of classification. This will ensure that we can identify strength and weakness of our KNN model, and eventually come up with a way to strengthen the model, by adding more data, features or etc.

Although we have mentioned KNN as a main classifier model, we can also try decision tree algorithm as a classifier, since decision tree may help us to come up with a certain threshold for each important features, instead of finding the closet matches between an NBA star and non-NBA star, which may be useful for us in predicting a NBA star.

To conclude, we will be mainly using machine learning classification algorithm as a primary way to come up with the prediction, instead of other types of machine learning algorithm such as regression or clustering. We will test each of these algorithms to see which works best on our dataset. 